# Week 3 Assignment
## Part 1: making table (10 marks)

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup


### Reading dataset from wikipedia page

In [2]:
#df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)[0]
#df.head()

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table), header=0)[0]
#df = pd.DataFrame(df)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [3]:
len(df)

289

### Removing rows with 'Not assigned' Boroughs

In [4]:
df = df[df['Borough'] != 'Not assigned'].reset_index(drop=True)
print(len(df))

212


77 rows has been removed!

In [5]:
df.head()


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [6]:
gdf = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ', '.join(x)).reset_index()
gdf.head()


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Next, we Check which neighbourhoods are 'Not assigned' and try renaming them to their boroughs

In [7]:
ind = gdf[gdf['Neighbourhood'] == 'Not assigned'].index.values
gdf.iloc[ind]

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


In [8]:
for i in ind:
    gdf.at[i,'Neighbourhood'] = gdf.at[i,'Borough']

In [9]:
gdf.iloc[ind]

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [10]:
gdf["Borough"].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

In [11]:
gdf.shape

(103, 3)

## Part 2: Adding Latitude and Longitude values of each Postal code (2 marks)

In [12]:
lat_log = pd.read_csv('http://cocl.us/Geospatial_data')
lat_log.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We must firt rename the column Postal Code to Postcode in order to merge them in one table

In [13]:
lat_log.rename(columns={'Postal Code':'Postcode'}, inplace=True)

In [14]:
toronto = pd.merge(gdf, lat_log, how='inner', on=['Postcode'])

In [15]:
len(toronto)

103

In [16]:
toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Part 3: Exploring and Clustring Neighbourhoods in Toronto (3 marks)

Let's get the geographical coordinates of Toronto.

In [18]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    certifi-2018.10.15         |        py36_1000         138 KB  conda-forge
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0            conda-forge
    geopy:           

In [19]:
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [20]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    mkl_fft-1.0.6              |           py36_0         586 KB  conda-forge
    mkl_random-1.0.2           |           py36_0         1.3 MB  conda-forge
    mkl-2018.0.3               |                1       198.7 MB
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    numpy-1.15.0               |   py36h1b885b7_0          35 KB
    branca-0.3.1               |             py_0          25 KB  conda-forge
    numpy-base-1.15.0          |   py36h3dfced4_0         4.2 MB
    pandas-0.23.4              |   py36hf8a1672_0        27.8 MB  conda-forge
    altair-2.2.2               |        py36_1001         494 KB  conda-forge
    folium-0.5.0               |             py_0  

In [21]:
# create map of Tronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto['Latitude'], toronto['Longitude'], toronto['Postcode']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define Foursquare Credentials and Version

In [22]:
CLIENT_ID = 'E2G4GBWGYJ1CRVHUJWYSKSBDJASMAK023KTXDB1U2Z4MULMU' # your Foursquare ID
CLIENT_SECRET = 'X4WBHPV1YT5ORIL1JGRNPQZMNDONH1Q5ZUMJCGNKFSNIJ1FB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: E2G4GBWGYJ1CRVHUJWYSKSBDJASMAK023KTXDB1U2Z4MULMU
CLIENT_SECRET:X4WBHPV1YT5ORIL1JGRNPQZMNDONH1Q5ZUMJCGNKFSNIJ1FB


In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Explore Neighborhoods in Toronto
Let's create a function to get top 100 venues for each postal code in Toronto

In [24]:
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Region', 
                  'Region Latitude', 
                  'Region Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
# type your answer here
neighborhoods = toronto.Postcode
latitudes = toronto.Latitude
longitudes = toronto.Longitude

toronto_venues = getNearbyVenues(neighborhoods, latitudes, longitudes, radius=800)

In [27]:
print(toronto_venues.shape)
toronto_venues.head()

(3941, 7)


,Region,Region Latitude,Region Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa
1,M1B,43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
2,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
3,M1B,43.806686,-79.194353,Tim Hortons,43.802000,-79.198169,Coffee Shop
4,M1B,43.806686,-79.194353,Harvey's,43.800106,-79.198258,Fast Food Restaurant


In [28]:
toronto_venues.groupby('Region').count()

,Region Latitude,Region Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Region,,,,,,
M1B,12,12,12,12,12,12
M1C,5,5,5,5,5,5
M1E,15,15,15,15,15,15
M1G,4,4,4,4,4,4
M1H,21,21,21,21,21,21
M1J,8,8,8,8,8,8
M1K,21,21,21,21,21,21
M1L,16,16,16,16,16,16
M1M,8,8,8,8,8,8


In [29]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 326 uniques categories.


## Analyzing each Region with unique a Postal Code

In [30]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add Region column back to dataframe
toronto_onehot['Region'] = toronto_venues['Region'] 

# move Region column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Region,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
toronto_onehot.shape

(3941, 327)

In [32]:
toronto_grouped = toronto_onehot.groupby('Region').mean().reset_index()
toronto_grouped.head()

,Region,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,M1C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,M1E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,M1G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,M1H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.047619


In [33]:
toronto_grouped.shape

(102, 327)

In [34]:
num_top_venues = 5

for hood in toronto_grouped['Region']:
    #print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Region'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 4})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')

### Let's put that into a pandas dataframe
First, let's write a function to sort the venues in descending order.

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Region']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
regions_venues_sorted = pd.DataFrame(columns=columns)
regions_venues_sorted['Region'] = toronto_grouped['Region']

for ind in np.arange(toronto_grouped.shape[0]):
    regions_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

regions_venues_sorted.head()

,Region,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M1B,Fast Food Restaurant,Coffee Shop,Spa
1,M1C,Breakfast Spot,Italian Restaurant,Burger Joint
2,M1E,Pizza Place,Coffee Shop,Fast Food Restaurant
3,M1G,Coffee Shop,Park,Business Service
4,M1H,Coffee Shop,Bakery,Indian Restaurant


In [37]:
regions_venues_sorted.describe()

,Region,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
count,102,102,102,102
unique,102,36,40,52
top,M4G,Coffee Shop,Coffee Shop,Café
freq,1,32,19,11


### Cluster Neighborhoods
Run k-means to cluster the neighborhood into 5 clusters.

In [75]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Region', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:5] 

array([2, 2, 1, 2, 2], dtype=int32)

In [39]:
len(kmeans.labels_)


102

Let's create a new dataframe that includes the cluster as well as the top 5 venues for each Region. First, we need to drop regions where there is no venues around them!

In [76]:
toronto_new = toronto[toronto['Postcode'].isin(regions_venues_sorted['Region'])]

In [77]:
toronto_merged = toronto_new

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each Region
toronto_merged.rename(columns={"Postcode":"Region"}, inplace=True)
toronto_merged = toronto_merged.join(regions_venues_sorted.set_index('Region'), on='Region')

toronto_merged.head() # check the last columns!

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Pop these, since the values are in `kwargs` under different names


,Region,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2,Fast Food Restaurant,Coffee Shop,Spa
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,2,Breakfast Spot,Italian Restaurant,Burger Joint
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Pizza Place,Coffee Shop,Fast Food Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2,Coffee Shop,Park,Business Service
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2,Coffee Shop,Bakery,Indian Restaurant


In [42]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [78]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Region'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [90]:
rainbow

['#8000ff', '#2adddd', '#d4dd80', '#ff0000']

## 5. Examine Clusters

### Cluster 1

In [79]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head(5)

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
20,North York,0,Cafeteria,Deli / Bodega,Pool
23,North York,0,Intersection,Bowling Alley,Bank
31,North York,0,Park,Coffee Shop,Shopping Mall
48,Central Toronto,0,Grocery Store,Park,Gym / Fitness Center
50,Downtown Toronto,0,Bank,Park,Trail


Cluster 1 is about sport and exercise because we see that some words like 'park', 'pool' and 'gym' are more frequent in this cluster.

In [92]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head(15)

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
2,Scarborough,1,Pizza Place,Coffee Shop,Fast Food Restaurant
5,Scarborough,1,Ice Cream Shop,Coffee Shop,Fast Food Restaurant
7,Scarborough,1,Coffee Shop,Diner,Bus Line
8,Scarborough,1,Pizza Place,Ice Cream Shop,Furniture / Home Store
13,Scarborough,1,Pizza Place,Pharmacy,Intersection
14,Scarborough,1,Chinese Restaurant,Pharmacy,Park
17,North York,1,Pharmacy,Park,Housing Development
24,North York,1,Coffee Shop,Grocery Store,Pharmacy
25,North York,1,Bus Stop,Furniture / Home Store,Road
28,North York,1,Coffee Shop,Pizza Place,Grocery Store


Cluster 2 is about pizza and fastfood because we see that some words like 'pizza place' and 'fastfood restaurant' are more frequent in this cluster.

In [91]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head(15)

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Scarborough,2,Fast Food Restaurant,Coffee Shop,Spa
1,Scarborough,2,Breakfast Spot,Italian Restaurant,Burger Joint
3,Scarborough,2,Coffee Shop,Park,Business Service
4,Scarborough,2,Coffee Shop,Bakery,Indian Restaurant
6,Scarborough,2,Discount Store,Coffee Shop,Grocery Store
9,Scarborough,2,Thai Restaurant,Convenience Store,College Stadium
10,Scarborough,2,Electronics Store,Furniture / Home Store,Indian Restaurant
11,Scarborough,2,Grocery Store,Middle Eastern Restaurant,Pizza Place
12,Scarborough,2,Chinese Restaurant,American Restaurant,Sandwich Place
15,Scarborough,2,Coffee Shop,Fast Food Restaurant,Chinese Restaurant


Cluster 3 is a mixture of different kinds of restaurants and Coffe Shops because of words like '*** Restaurant' and 'Coffe Shop'.

In [82]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head(5)

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
102,Etobicoke,3,Lounge,Yoga Studio,Donut Shop


Cluster 4 is an outlier.

In [85]:
#toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head(5)

In [84]:
#toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head(5)

In [86]:
#toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head(5)